In [ ]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-bright')

np.random.seed(0)
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
print('Train Data Shape: {}'.format(train.shape))
print('Test Data Shape: {}'.format(test.shape))

In [ ]:
train.head()

#### Label Samples Distribution

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(train['label'])
plt.xlabel('Label')
plt.ylabel('Count')

In [ ]:
y_train = train.label.astype('float32')
X_train = train.drop('label', axis=1).astype('float32')
X_test = test.astype('float32')

print(X_train.shape, y_train.shape, X_test.shape)

#### Reshaping Data

In [ ]:
X_train = X_train.values.reshape(-1, 28, 28, 1)
X_train = X_train / 255.0

X_test = X_test.values.reshape(-1, 28, 28, 1)
X_test = X_test / 255.0

print(X_train.shape, X_test.shape)

##### Lets Take a Look at Some Samples from both Datasets

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(1, 6):
    plt.subplot(5, 5, i)
    plt.imshow(X_train[random.randint(0, len(X_train))])

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(1, 6):
    plt.subplot(5, 5, i)
    plt.imshow(X_test[random.randint(0, len(X_test))])

#### One Hot Encoding Target Values

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_train.shape

##### Here is the comparison between standart and encoded values

In [ ]:
train['label'].head()

In [ ]:
y_train[0:5, :]

### Modeling

In [ ]:
model = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)), 
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2), 
    Conv2D(64, (3, 3), activation='relu', padding='Same'), 
    Conv2D(64, (3, 3), activation='relu', padding='Same'), 
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)), 
    Dropout(0.25), 
    Conv2D(64, (3, 3), activation='relu', padding='Same'), 
    Conv2D(64, (3, 3), activation='relu', padding='Same'), 
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Dropout(0.25), 
    Flatten(), 
    Dense(256, activation='relu'), 
    Dense(256, activation='relu'), 
    Dropout(0.50), 
    Dense(10, activation='softmax')
])
model.summary()

##### Class to stop fitting when it needs

In [ ]:
class Callback(tf.keras.callbacks.Callback):
    def end_if(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.999):
            print('Reached 0.99 accuracy')
            self.model.stop_training = True
            
callbacks = Callback()

#### Fitting the model

In [ ]:
optimizer = tf.keras.optimizers.Adam(
                    learning_rate=0.0005, 
                    beta_1=0.9, 
                    beta_2=0.999, 
                    epsilon=1e-07, 
                    name='Adam')

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=50, epochs=20, callbacks=[callbacks])

In [ ]:
pred = model.predict(X_test)

pred = np.argmax(pred, axis=1)

pred = pd.Series(pred, name='Label')

In [ ]:
submission = pd.concat([pd.Series(range(1, 28001), name='ImageID'), pred], axis=1)
submission.to_csv('submission.csv', index=False)

submission.head(10)